### 分析bigquant的比特币策略回测结果

In [ ]:
# %%

import pyfolio as pf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import json
import warnings
warnings.filterwarnings('ignore')
bitcoin_strategy_backtest = pd.read_csv('bitcoin_strategy_backtest.csv', index_col=0, parse_dates=True)
returns = bitcoin_strategy_backtest['returns']
# print(returns.head())
print(pf.__version__)
#plt.show()

bitcoin_strategy_backtest['positions'] = bitcoin_strategy_backtest['positions'].apply(lambda x: x.replace("'", '"'))
bitcoin_strategy_backtest['transactions'] = bitcoin_strategy_backtest['transactions'].apply(lambda x: x.replace("'", '"'))
bitcoin_strategy_backtest['transactions'] = bitcoin_strategy_backtest['transactions'].apply(lambda x: x.replace("datetime.", '"datetime.'))
bitcoin_strategy_backtest['transactions'] = bitcoin_strategy_backtest['transactions'].apply(lambda x: x.replace(")", ')"'))
positions = bitcoin_strategy_backtest[['positions', 'ending_cash']]
position_data = json.loads(positions.iloc[1, 0])
symbol = position_data[0]['symbol']
positions_edited = pd.DataFrame(0, index = positions.index, columns = [[symbol, 'cash']])
for index, row in positions.iterrows():
    position_data = json.loads(row['positions'])
    if position_data:
        positions_edited.loc[index, symbol] = position_data[0]['market_value']   
    positions_edited.loc[index, 'cash'] = int(row['ending_cash'])
#print(positions_edited.head()) 

transactions = bitcoin_strategy_backtest[['transactions']]
# print('transactions: ', transactions.head())

transactions_edited = pd.DataFrame(np.nan, index = transactions.index, columns = [['amount', 'price', 'symbol']])
# print('transactions_edited: ', transactions_edited.head())

transactions_edited['symbol'] = transactions_edited['symbol'].astype(str)
for index, row in transactions.iterrows():
    transactions_data = json.loads(row['transactions'])
    if transactions_data:
        transactions_edited.loc[index, 'price'] = float(transactions_data[0]['price'])   
        transactions_edited.loc[index, 'amount'] = transactions_data[0]['amount']           
        transactions_edited.loc[index, 'symbol'] = symbol
    else:
        transactions_edited.loc[index, 'price'] = 0
        transactions_edited.loc[index, 'amount'] = 0
        transactions_edited.loc[index, 'symbol'] = symbol   
# transactions_edited = transactions_edited.dropna()
transactions_edited = transactions_edited[(transactions_edited['amount'] != 0) & (~transactions_edited['price'].isna())]
print('returns: ', returns.head())
print('positions_edited: ', positions_edited.head())
print('transactions_edited: ', transactions_edited.head())
# pf.create_returns_tear_sheet(returns,
#                          positions=positions_edited,
#                          transactions=transactions_edited)
#                          slippage=1)
# help(pf.create_full_tear_sheet)
pf.create_full_tear_sheet(returns,
                         positions=positions_edited,
                        transactions=transactions_edited,
                        round_trips=True,
                        bootstrap=False,
                        estimate_intraday=False,
                        slippage=10)
plt.show()

: 

### 分析freqtrade的回测结果

In [1]:
# %%

import pyfolio as pf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import json
import warnings
warnings.filterwarnings('ignore')
returns_df = pd.read_csv('returns.csv', parse_dates=True)
returns_df = returns_df.dropna()
returns_df['date'] = pd.to_datetime(returns_df['date'])
returns_df.set_index('date', inplace=True)
returns = returns_df['returns']
# 严格去除 returns 中的 NaN
returns = returns.dropna()
# 确保 index 为 datetime 类型，且已排序
returns.index = pd.to_datetime(returns.index)
returns = returns.sort_index()
# 去除重复 index（如有）
returns = returns[~returns.index.duplicated(keep='first')]
# 检查是否还有 NaT
print(returns.index[returns.index.isna()])
print(returns.isna().sum())
positions = pd.read_csv('positions.csv', parse_dates=True)

positions['date'] = pd.to_datetime(positions['date'])
positions.set_index('date', inplace=True)

transactions = pd.read_csv('transactions.csv', parse_dates=True)
transactions['date'] = pd.to_datetime(transactions['date'])
transactions.set_index('date', inplace=True)
print(pf.__version__)
#print(returns.dtypes)
#print(positions.dtypes)
#print(transactions.dtypes)
#print('returns: ', returns.head())
#print('returns: ', returns.tail())
print(returns.head(10))
print(returns.tail(10))
print(returns.index)
print("唯一值：", returns.unique())
print("非零天数：", (returns != 0).sum())
print("全部为零？", (returns == 0).all())
print("最大值：", returns.max())
print("最小值：", returns.min())
#print('positions: ', positions.head())
#print('positions: ', positions.tail())
#print('transactions: ', transactions.head())
#print('transactions: ', transactions.tail())
pf.create_returns_tear_sheet(returns,
                          positions=positions,
                          transactions=transactions)
#                          slippage=1)
# help(pf.create_full_tear_sheet)
#pf.create_full_tear_sheet(returns,
#                         positions=positions,
#                        transactions=transactions,
#                        round_trips=True,
#                        bootstrap=False,
#                        estimate_intraday=False,
#                        slippage=10)
plt.show()

ModuleNotFoundError: No module named 'pyfolio'